In [1]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup
import time
import json
from joblib import Parallel, delayed
import re
import numpy as np
import pandas_profiling

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_selection import f_regression, mutual_info_regression
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, make_scorer
from catboost import CatBoostRegressor
import xgboost as xgb


In [2]:
random_seed = 42  # фиксируем RANDOM_SEED для воспроизводимости результатов
val_size = 0.20

test = pd.read_csv('test.csv')
data = pd.read_csv('cars.csv')

In [3]:
test.head()

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,...,LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,...,LIFTBACK ROBOT 1.8,роботизированная,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
3,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,коричневый,"{""id"":""20803582"",""name"":""Ambition"",""available_...",КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6 LTR,110 N12,"{""cruise-control"":true,""roller-blind-for-rear-...",бензин,...,LIFTBACK AUTOMATIC 1.6,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
4,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,белый,NaN,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8 LTR,152 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK AUTOMATIC 1.8,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


In [4]:
data.head()

,body_type,brand,color,complectation_dict,condition,customs,description,engine_displacement,engine_power,equipment_dict,...,price,production_date,super_gen,tenure,vehicle_passport,vehicle_transmission,vendor,wheel,wheel_drive,fuel_type
0,LIFTBACK,SKODA,коричневый,"{""id"":""0"",""name"":""""}",Не требует ремонта,Растаможен,Не тратьте время на поиски. Уже сегодня Вы смо...,1.5 LTR,150 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",...,1600000.0,2017.0,"{""id"":""20898195"",""name"":""III (A7) Рестайлинг"",...",NaN,ORIGINAL,ROBOT,EUROPEAN,LEFT,передний,GASOLINE
1,LIFTBACK,SKODA,белый,"{""id"":""0"",""name"":""""}",Не требует ремонта,Растаможен,Не такси! Пoкупка и эксплуатация с января 2017...,1.2 LTR,110 N12,{},...,830000.0,2016.0,"{""id"":""9275797"",""name"":""I"",""ru_name"":""1"",""year...",4 года и 10 месяцев,ORIGINAL,MECHANICAL,EUROPEAN,LEFT,передний,GASOLINE
2,COMPACTVAN,SKODA,чёрный,"{""id"":""7815321"",""name"":""Ambition"",""available_o...",Не требует ремонта,Растаможен,- Автомобиль в отличном состоянии.\n- За все...,1.6 LTR,90 N12,"{""alloy-wheel-disks"":true,""tinted-glass"":true,...",...,730000.0,2012.0,"{""id"":""6412280"",""name"":""I Рестайлинг"",""ru_name...",NaN,ORIGINAL,MECHANICAL,EUROPEAN,LEFT,передний,DIESEL
3,LIFTBACK,SKODA,зелёный,"{""id"":""0"",""name"":""""}",Не требует ремонта,Растаможен,"Для своих лет,бодрая,торг у капота!снята с учё...",2.0 LTR,115 N12,{},...,180000.0,1998.0,"{""id"":""5140407"",""name"":""I (A4)"",""ru_name"":""1 (...",NaN,DUPLICATE,AUTOMATIC,EUROPEAN,LEFT,передний,GASOLINE
4,ALLROAD_5_DOORS,SKODA,коричневый,"{""id"":""6038962"",""name"":""Active"",""available_opt...",Не требует ремонта,Растаможен,"Двигатель, турбина, коробка, подвеска без наре...",2.0 LTR,140 N12,"{""cruise-control"":true,""esp"":true,""airbag-driv...",...,460000.0,2010.0,"{""id"":""6038909"",""name"":""I"",""ru_name"":""1"",""year...",NaN,DUPLICATE,ROBOT,EUROPEAN,LEFT,полный,DIESEL


In [5]:
test.columns

Index(['bodyType', 'brand', 'car_url', 'color', 'complectation_dict',
       'description', 'engineDisplacement', 'enginePower', 'equipment_dict',
       'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name',
       'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency',
       'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration',
       'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС',
       'Привод', 'Руль', 'Состояние', 'Таможня'],
      dtype='object')

In [6]:
#приводим столбцы к одинаковому формату
columns = ['body_type', 'brand', 'car_url', 'color', 'complectation_dict',
       'description', 'engine_displacement', 'engine_power', 'equipment_dict',
       'fuel_type', 'image', 'mileage', 'model_date', 'model_info', 'model_name',
       'name', 'number_of_doors', 'parsing_unixtime', 'price_currency',
       'production_date', 'sell_id', 'super_gen', 'vehicle_configuration',
       'vehicle_transmission', 'vendor', 'owners', 'tenure', 'vehicle_passport',
       'wheel_drive', 'wheel', 'condition', 'customs']

In [7]:
#присваиваем новые названия
test.columns = columns

In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   body_type              34686 non-null  object
 1   brand                  34686 non-null  object
 2   car_url                34686 non-null  object
 3   color                  34686 non-null  object
 4   complectation_dict     6418 non-null   object
 5   description            34686 non-null  object
 6   engine_displacement    34686 non-null  object
 7   engine_power           34686 non-null  object
 8   equipment_dict         24690 non-null  object
 9   fuel_type              34686 non-null  object
 10  image                  34686 non-null  object
 11  mileage                34686 non-null  int64 
 12  model_date             34686 non-null  int64 
 13  model_info             34686 non-null  object
 14  model_name             34686 non-null  object
 15  name               

Значения в одинаковых столбцах двух датафреймов на разных языках. В test переведем значения на английский

In [9]:
body_type_dic = {'седан': 'SEDAN',
 'внедорожник 5 дв.': 'ALLROAD_5_DOORS',
 'лифтбек': 'LIFTBACK',
 'хэтчбек 5 дв.': 'HATCHBACK_5_DOORS',
 'универсал 5 дв.': 'WAGON_5_DOORS',
 'минивэн': 'MINIVAN',
 'купе': 'COUPE',
 'компактвэн': 'COMPACTVAN',
 'хэтчбек 3 дв.': 'HATCHBACK_3_DOORS',
 'пикап двойная кабина': 'PICKUP_TWO',
 'купе-хардтоп': 'COUPE_HARDTOP',
 'внедорожник 3 дв.': 'ALLROAD_3_DOORS',
 'родстер': 'ROADSTER',
 'фургон': 'VAN',
 'кабриолет': 'CABRIO',
 'седан-хардтоп': 'SEDAN_HARDTOP',
 'микровэн': 'MICROVAN',
 'лимузин': 'LIMOUSINE',
 'седан 2 дв.': 'SEDAN_2_DOORS',
 'пикап полуторная кабина': 'PICKUP_FULL',
 'пикап одинарная кабина': 'PICKUP_MEAN',
 'внедорожник открытый': 'ALLROAD_OPEN',
 'тарга': 'TARGA',
 'фастбек': 'FUSTBACK'}

In [10]:
test['body_type'] = test['body_type'].map(body_type_dic)

In [11]:
test['vehicle_passport'] = test['vehicle_passport'].map({'Оригинал':'ORIGINAL', 'Дубликат':'DUPLICATE'})

In [12]:
test['vehicle_transmission'] = test['vehicle_transmission'].map({'автоматическая':'AUTOMATIC', 'механическая':'MECHANICAL', 
                                                                 'вариатор':'VARIATOR', 'роботизированная':'ROBOT'})

In [13]:
test['wheel'] = test['wheel'].map({'Левый':'LEFT', 'Правый':'RIGHT'})

In [14]:
test['fuel_type'] = test['fuel_type'].map({'бензин':'GASOLINE', 'дизель':'DIESEL', 
                                            'гибрид':'HYBRID', 'электро':'ELECTRO', 'газ':'LPG'})

Посмотрим сколько значений в столбце 'price' равны нулю

In [15]:
data[data['price']==0]['price'].value_counts()

0.0    4515
Name: price, dtype: int64

Строки с нулевой ценой не имеют ценности для модели. Не включаем их в датафрейм

In [16]:
data = data[data['price']>0]

In [17]:
# в тесте нет столбца 'price' добавляем его
test['price'] = 0

In [18]:
test = test[data.columns]

In [19]:
#объединяем датафреймы
data['sample'] = 1
test['sample'] = 0
df = data.append(test, sort=False).reset_index(drop=True)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75239 entries, 0 to 75238
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   body_type             75239 non-null  object 
 1   brand                 75239 non-null  object 
 2   color                 75239 non-null  object 
 3   complectation_dict    46971 non-null  object 
 4   condition             65721 non-null  object 
 5   customs               65669 non-null  object 
 6   description           75239 non-null  object 
 7   engine_displacement   75239 non-null  object 
 8   engine_power          75239 non-null  object 
 9   equipment_dict        65243 non-null  object 
 10  mileage               75239 non-null  float64
 11  model_info            75239 non-null  object 
 12  model_name            75239 non-null  object 
 13  number_of_doors       75239 non-null  float64
 14  owners                75239 non-null  object 
 15  price              

В некоторых столбцах много пропусков. Нужно их заполнить

In [21]:
df['complectation_dict'].value_counts()

{"id":"0","name":""}                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

В столбце 'complectation_dict' 28268 пропусков и 24432 не имеющих смысла значений {"id":"0","name":""}. Данных слишком мало, можно удалить столбец

In [22]:
df = df.drop(['complectation_dict'], axis = 1)

In [23]:
df['condition'].value_counts()

Не требует ремонта    65721
Name: condition, dtype: int64

In [24]:
df['customs'].value_counts()

Растаможен    65669
Name: customs, dtype: int64

Столбцы 'customs', 'condition' имеют только по одному значению, поэтому их можно удалить

In [25]:
df = df.drop(['customs', 'condition'], axis = 1)

В столбце 'equipment_dict' пустые значения заменим выражением "нет данных"

In [26]:
df['equipment_dict'] = df['equipment_dict'].fillna('n')
for index in df.index:
    if df.loc[index, 'equipment_dict'] == '{}':
        df.loc[index, 'equipment_dict'] = 'n'

В столбце 'tenure' мало информации. Корректно заполнить его невозможно, поэтому удаляем данный столбец

In [27]:
df = df.drop(['tenure'], axis = 1)

Столбец 'vehicle_passport' имеет мало пропущенных значений, поэтому заполним чаще встречающимся

In [28]:
df['vehicle_passport'].value_counts()

ORIGINAL     63058
DUPLICATE    11008
Name: vehicle_passport, dtype: int64

In [29]:
df['vehicle_passport'] = df['vehicle_passport'].fillna('ORIGINAL')

В столбце 'wheel_drive' пропущенные значения заполним типом привода аналогичных моделей в других строках.

In [30]:
# создаем словарь, где ключи - это модель автомобиля, а значения - тип привода
wheel_drive_dict = {}
for elem in df[df['wheel_drive'].isnull() == False].index:
    wheel_drive_dict[df.loc[elem, 'model_name']] = df.loc[elem, 'wheel_drive']

In [31]:
# проверяем каких моделей нет в словаре
for elem in df[df['wheel_drive'].isnull()]['model_name'].value_counts().index:
    if elem not in wheel_drive_dict:
        print(elem)

Q5_SPORTBACK
QX55


In [32]:
# присваиваем отсутствующим ключам значения
wheel_drive_dict['Q5_SPORTBACK'] = 'полный'
wheel_drive_dict['QX55'] = 'полный'

In [33]:
# каждую ячейку с пропущенным значением заполняем значением ключа, который соответствует модели автомобиля в этой строке  
for elem in df[df['wheel_drive'].isnull()].index:
    df.loc[elem, 'wheel_drive'] = wheel_drive_dict[df.loc[elem, 'model_name']]

Смотрим у каких брендов автомобилей не указан поставщик, и вручную заполняем пропуски

In [34]:
df[df['vendor'].isnull()]['brand'].value_counts()

TOYOTA    205
Name: brand, dtype: int64

In [35]:
df['vendor'] = df['vendor'].fillna('JAPANESE')

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75239 entries, 0 to 75238
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   body_type             75239 non-null  object 
 1   brand                 75239 non-null  object 
 2   color                 75239 non-null  object 
 3   description           75239 non-null  object 
 4   engine_displacement   75239 non-null  object 
 5   engine_power          75239 non-null  object 
 6   equipment_dict        75239 non-null  object 
 7   mileage               75239 non-null  float64
 8   model_info            75239 non-null  object 
 9   model_name            75239 non-null  object 
 10  number_of_doors       75239 non-null  float64
 11  owners                75239 non-null  object 
 12  price                 75239 non-null  float64
 13  production_date       75239 non-null  float64
 14  super_gen             75239 non-null  object 
 15  vehicle_passport   

In [37]:
# посмотрим количество значений в столбцах
for column in df.columns:
    print(column, ':', len(df[column].value_counts()))

body_type : 24
brand : 12
color : 16
description : 33133
engine_displacement : 55
engine_power : 324
equipment_dict : 25002
mileage : 11272
model_info : 1191
model_name : 557
number_of_doors : 5
owners : 8
price : 575
production_date : 70
super_gen : 7171
vehicle_passport : 2
vehicle_transmission : 4
vendor : 2
wheel : 2
wheel_drive : 3
fuel_type : 5
sample : 2


In [38]:
# создадим список бинарных столбцов
bin_cols = []
for column in df.columns:
    if len(df[column].value_counts()) == 2:
        bin_cols.append(column)
print(bin_cols)

['vehicle_passport', 'vendor', 'wheel', 'sample']


In [39]:
bin_cols.remove('sample')
label_encoder = LabelEncoder()

for column in bin_cols:
    df[column] = label_encoder.fit_transform(df[column])

In [40]:
df[bin_cols].head()

,vehicle_passport,vendor,wheel
0,1,0,0
1,1,0,0
2,1,0,0
3,0,0,0
4,0,0,0


Столбцы, у которых значечния количественные, переведем в числа

In [41]:
# занчения столбца с описаниями заменим на длину описаний
df['description'] = df['description'].apply(lambda x: len(x))

In [42]:
# в 'engine_displacement' оставляем только числа
df['engine_displacement'] = df['engine_displacement'].apply(lambda x: float(re.sub(r'\sLTR', '', x)) if len(x)>4 else 0)

In [43]:
df['engine_displacement']

0        1.5
1        1.2
2        1.6
3        2.0
4        2.0
        ... 
75234    1.6
75235    2.0
75236    2.5
75237    2.0
75238    3.0
Name: engine_displacement, Length: 75239, dtype: float64

In [44]:
df['engine_power'].value_counts().head()

150 N12    4860
249 N12    3092
110 N12    2636
140 N12    2201
105 N12    1898
Name: engine_power, dtype: int64

In [45]:
# в 'engine_power' оставляем только числа
df['engine_power'] = df['engine_power'].apply(lambda x: float(re.sub(r'\sN12', '', x)))

In [46]:
df['engine_power'].value_counts().head()

150.0    4860
249.0    3092
110.0    2636
140.0    2201
105.0    1898
Name: engine_power, dtype: int64

In [47]:
df['owners'].value_counts().head()

3 или более    15957
4              12406
0               9518
1 владелец      9459
2 владельца     9270
Name: owners, dtype: int64

In [48]:
# в 'owners' оставляем только числа
df['owners'] = df['owners'].apply(lambda x: int(re.sub(r'\D', '', x)) if type(x) == str else x)

In [49]:
df['owners'].value_counts().head()

3    20806
2    16672
1    15837
4    12406
0     9518
Name: owners, dtype: int64

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75239 entries, 0 to 75238
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   body_type             75239 non-null  object 
 1   brand                 75239 non-null  object 
 2   color                 75239 non-null  object 
 3   description           75239 non-null  int64  
 4   engine_displacement   75239 non-null  float64
 5   engine_power          75239 non-null  float64
 6   equipment_dict        75239 non-null  object 
 7   mileage               75239 non-null  float64
 8   model_info            75239 non-null  object 
 9   model_name            75239 non-null  object 
 10  number_of_doors       75239 non-null  float64
 11  owners                75239 non-null  int64  
 12  price                 75239 non-null  float64
 13  production_date       75239 non-null  float64
 14  super_gen             75239 non-null  object 
 15  vehicle_passport   

Столбцы 'model_info', 'super_gen' дублируют информацию из других столбцов. Удаляем их

In [51]:
df = df.drop(['model_info', 'super_gen'], axis=1)

In [52]:
cat_columns = ['body_type', 'brand', 'color', 'model_name', 'vehicle_transmission', 'wheel_drive', 'fuel_type']

In [53]:
# сделаем унитарное кодирование категориальных столбцов
df = pd.get_dummies(df, columns=cat_columns)

In [54]:
# создаем списки из строк в столбце 'equipment_dict'
for index in df.index:
    no_data_list = []
    if df.loc[index, 'equipment_dict']!='no data':
        df.loc[index, 'equipment_dict'] = re.sub(r':true', '', re.sub(r'{|}$', '', df.loc[index, 'equipment_dict']))
        
df['equipment_dict'] = df['equipment_dict'].apply(lambda x: x.split(',') if x!='n' else x)

In [55]:
# и так же сделаем унитарное кодирование
mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('equipment_dict')), index=df.index, columns=mlb.classes_))

In [56]:
df

,description,engine_displacement,engine_power,mileage,number_of_doors,owners,price,production_date,vehicle_passport,vendor,...,"""wheel-heat""","""wheel-leather""","""wheel-memory""","""wheel-power""","""windcleaner-heat""","""windscreen-heat""","""wireless-charger""","""xenon""","""ya-auto""",n
0,1588,1.5,150.0,56000.0,5.0,2,1600000.0,2017.0,1,0,...,0,1,0,0,0,1,0,0,0,0
1,360,1.2,110.0,153000.0,5.0,1,830000.0,2016.0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,1313,1.6,90.0,151000.0,5.0,2,730000.0,2012.0,1,0,...,0,0,0,1,0,0,0,0,0,0
3,109,2.0,115.0,318000.0,5.0,3,180000.0,1998.0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,175,2.0,140.0,297000.0,5.0,4,460000.0,2010.0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75234,414,1.6,136.0,115000.0,4.0,3,0.0,2014.0,1,0,...,1,1,0,0,0,0,0,1,0,0
75235,30,2.0,190.0,98000.0,4.0,1,0.0,2018.0,1,0,...,0,0,0,0,0,0,0,0,0,1
75236,17,2.5,170.0,360000.0,4.0,3,0.0,1997.0,0,0,...,0,0,0,0,0,0,0,0,0,1
75237,596,2.0,184.0,90500.0,5.0,2,0.0,2013.0,1,0,...,1,1,0,0,0,0,0,1,0,0


In [57]:
# по объявлениям 2021 года (тестовый датасет за 2020 год) 
# умножим 'price' на коэффициент 0.86

for index in df.index:
    if df.loc[index, 'price'] > 0:
        df.loc[index, 'price'] = df.loc[index, 'price']*0.86

In [58]:
# разделяем тестовую и обучающую выборки
X = df.query('sample == 1').drop(['price', 'sample'], axis=1).reset_index(drop=True)
X_sub = df.query('sample == 0').drop(['price', 'sample'], axis=1)
y = df.query('sample == 1').price.reset_index(drop=True)

In [60]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=val_size, shuffle=True, random_state=random_seed)

In [61]:
def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred)/y_true))

In [63]:
# задаем параметры модели
model_CBR = CatBoostRegressor(iterations = 5000,
                          random_seed = random_seed,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )

# проводим обучение
model_CBR.fit(X_train, np.log(y_train),
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         )

model_CBR.save_model('catboost_single_2_model.model')

# оцениваем точность
predict_test = np.exp(model_CBR.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 1.14%
